In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from ingest.catalog import DatasetCatalog
from ingest.loader import RawDatasetLoader
from ingest.fetch.csv import CsvAdapter
from ingest.fetch.sct import SocrataAdapter
from ingest.fetch.excel import ExcelAdapter
from ingest.fetch.metadata import SocrataMetadata

In [3]:
## servicios
catalog = DatasetCatalog()
loader = RawDatasetLoader(
    csv_adapter=CsvAdapter(),
    sct_adapter=SocrataAdapter(),
    excel_adapter=ExcelAdapter(),
)

In [4]:
ds = catalog.get("pib_municipios")
meta = loader.metadata(ds)
if isinstance(meta, SocrataMetadata):
    print("Nombre:", meta.name)
    print("Descripción:", meta.description)
    print("Columnas:")
    columns = meta.columns
    for col in columns:
        print(f"  - {col.name} ({col.fieldName}): {col.dataTypeName}")
else:
    print("Este origen no soporta metadata estructurada.")

Nombre: PIB Departamental con proyección
Descripción: Presenta la proyección del Producto Interno Bruto (PIB) constante, de los Departamentos de Colombia,  expresado en precios de 2015.
Columnas:
  - Año (a_o): number
  - Actividad (actividad): text
  - Sector (sector): text
  - Tipo de precios (tipo_de_precios): text
  - Código Departamento (DIVIPOLA) (c_digo_departamento_divipola): number
  - Departamento (departamento): text
  - Valor (miles de millones de pesos) (valor_miles_de_millones_de): number


In [5]:
records = list(loader.load(ds))
df = pd.DataFrame(records)
## filtrar por año 2015 y 2022 a 2025
df.head(3)

,a_o,actividad,sector,tipo_de_precios,c_digo_departamento_divipola,departamento,valor_miles_de_millones_de
0,2005,Construcción,Secundario,PIB a precios corrientes,97,Vaupés,2.28
1,2005,Construcción,Secundario,PIB a precios corrientes,99,Vichada,10.23
2,2013,Información y comunicaciones,Terciario,PIB a precios constantes de 2015,15,Boyacá,344.68


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16302 entries, 0 to 16301
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   a_o                           16302 non-null  object
 1   actividad                     16302 non-null  object
 2   sector                        16302 non-null  object
 3   tipo_de_precios               16302 non-null  object
 4   c_digo_departamento_divipola  16302 non-null  object
 5   departamento                  16302 non-null  object
 6   valor_miles_de_millones_de    16302 non-null  object
dtypes: object(7)
memory usage: 891.6+ KB


In [7]:
cols = [
    "a_o",
    "c_digo_departamento_divipola",
    "valor_miles_de_millones_de"
]

pretty_names = {
    col.fieldName: col.name
    for col in meta.columns
}

def pretty(col):
    return pretty_names.get(col, col)

# Convertir a número
for c in cols:
    df[c] = df[c].astype(str)\
                 .str.replace("$", "", regex=False)\
                 .str.replace(".", "", regex=False)\
                 .str.replace(",", ".", regex=False)\
                 .astype(float) 
    df[c] = pd.to_numeric(df[c], errors='coerce')


In [8]:
# filtrar 2015, 2022-2025
df = df[df["a_o"].isin([2015, 2022, 2023, 2024, 2025])].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2574 entries, 3414 to 16301
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   a_o                           2574 non-null   float64
 1   actividad                     2574 non-null   object 
 2   sector                        2574 non-null   object 
 3   tipo_de_precios               2574 non-null   object 
 4   c_digo_departamento_divipola  2574 non-null   float64
 5   departamento                  2574 non-null   object 
 6   valor_miles_de_millones_de    2574 non-null   float64
dtypes: float64(3), object(4)
memory usage: 160.9+ KB


In [9]:
## cuantos registros por año
df['a_o'].value_counts().sort_index()

a_o
2015.0    858
2022.0    858
2023.0    858
Name: count, dtype: int64